In [2]:
import json
from IPython.display import display, Markdown
import yaml

from pathlib import Path

import torch.nn as nn

from mymodel import init_my_model


model = init_my_model() 
min_max_yaml = yaml.safe_load(open("min_max.yaml", "r"))
config = json.load(open("config.json", "r"))
stats = yaml.safe_load(open("stats.yaml", "r"))

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

model_name = model.__class__.__name__
num_layers = len(list(model.net.children()))
num_epochs = stats["epochs"]
best_train_loss = stats["best_train_loss"]
best_test_loss = stats["best_test_loss"]
best_val_loss = stats["best_val_loss"]

mae_u_train = stats["metrics"]["train"]["u"]["mae"]
mae_v_train = stats["metrics"]["train"]["v"]["mae"]

mae_u_test = stats["metrics"]["test"]["u"]["mae"]
mae_v_test = stats["metrics"]["test"]["v"]["mae"]

mae_u_val = stats["metrics"]["validation"]["u"]["mae"]
mae_v_val = stats["metrics"]["validation"]["v"]["mae"]

rmse_u_train = stats['metrics']['train']['u']['rmse']
rmse_v_train = stats['metrics']['train']['v']['rmse']

rmse_u_test = stats['metrics']['test']['u']['rmse']
rmse_v_test = stats['metrics']['test']['v']['rmse']

rmse_u_val = stats['metrics']['validation']['u']['rmse']
rmse_v_val = stats['metrics']['validation']['v']['rmse']


In [3]:
md_text = "_This document was generated automatically after training._\n\n"
md_text += "# Model Architecture\n"
md_text += f"The following section describes the architecture of the {model_name} used for the submission. "
md_text += f"The {model_name} has a total number of {total_params} parameters "
md_text += f"with {trainable_params} trainable parameters. "
md_text += f"It consists of {num_layers} layers. "
md_text += "The layers are structured as follows:\n\n"
md_text += "| Layer Type | Details |\n"
md_text += "| :--- | :--- |\n"

def get_details(layer):
    # This might be very incomplete :)
    attr_map = {
        'in_channels': 'In', 'out_channels': 'Out',
        'in_features': 'In', 'out_features': 'Out',
        'kernel_size': 'Kernel', 'stride': 'Stride', 'padding': 'Padding',
        'p': 'Drop', 'negative_slope': 'Slope', 'alpha': 'Alpha',
        'lambd': 'Lambda',
        'beta': 'Beta',               # Softplus, Swish
        'threshold': 'Threshold',     # Hardshrink, Softshrink, Threshold
        'value': 'Value',             # Threshold
        'min_val': 'Min',             # Hardtanh
        'max_val': 'Max',             # Hardtanh
        'lower': 'Lower',             # RReLU
        'upper': 'Upper',             # RReLU
        'approximate': 'Approximate', # GELU
        'num_parameters': 'Params'    # PReLU
    }
    
    parts = []
    for attr, label in attr_map.items():
        if hasattr(layer, attr):
            val = getattr(layer, attr)
            # Format tuples like (3, 3) into "3x3"
            if isinstance(val, (tuple, list)):
                val = "x".join(map(str, val))
            parts.append(f"{label}: {val}")
            
    return ", ".join(parts) if parts else "---"

for layer in model.net.children():
    layer_type = layer.__class__.__name__
    details = get_details(layer)
    md_text += f"| {layer_type} | {details} |\n"

display(Markdown(md_text))

_This document was generated automatically after training._

# Model Architecture
The following section describes the architecture of the FluidCNN used for the submission. The FluidCNN has a total number of 98802 parameters with 98802 trainable parameters. It consists of 9 layers. The layers are structured as follows:

| Layer Type | Details |
| :--- | :--- |
| Conv2d | In: 1, Out: 16, Kernel: 11x11, Stride: 1x1, Padding: same |
| ELU | Alpha: 1.0 |
| Conv2d | In: 16, Out: 16, Kernel: 11x11, Stride: 1x1, Padding: same |
| ELU | Alpha: 1.0 |
| Conv2d | In: 16, Out: 16, Kernel: 11x11, Stride: 1x1, Padding: same |
| ELU | Alpha: 1.0 |
| Conv2d | In: 16, Out: 16, Kernel: 11x11, Stride: 1x1, Padding: same |
| ELU | Alpha: 1.0 |
| Conv2d | In: 16, Out: 2, Kernel: 11x11, Stride: 1x1, Padding: same |


In [4]:
# Mapping of keys to their ML explanations
EXPLANATIONS = {
    "activation": "Activation function.",
    "model": "Model Class Name.",
    "train_ratio": "Training/Validation split proportion.",
    "learning_rate": "Optimizer step size.",
    "weight_decay": "Regularization coefficient.",
    "use_lr_scheduler": "Dynamic LR adjustment per epoch.",
    "use_early_stopping": "Stop when model stops converging.",
    "early_stopping_patience": "Epochs to wait for improvement.",
    "criterion": "Loss function objective.",
    "batch_size": "Training data in one iteration.",
    "epochs": "Total Number of Epochs.",
    "num_hidden_layers": "Network depth.",
    "kernel_size": "The size of the kernel.",
    "in_channels": "Input feature map depth.",
    "out_channels": "Number of learned filters.",
    "hidden_channels": "Depth of the Network.",
    "output_activation": "Final layer activation.",
    "use_bias": "Learnable additive offset.",
    "padding_mode": "Boundary condition strategy.",
    "random_split": "Whether to split data randomly.",
    "random_seed": "Torch Seed.",
}

md_text = "This was the configuration used to initialize the model and load the data:\n\n"
md_text += "| Parameter | Value | Explanation |\n"
md_text += "| :--- | :--- | :--- |\n"

for k, v in sorted(config.items()):
    display_key = k.replace("_", " ").title()
    explanation = EXPLANATIONS.get(k, "---")
    md_text += f"| {display_key} | {v} | {explanation} |\n"

display(Markdown(md_text))

This was the configuration used to initialize the model and load the data:

| Parameter | Value | Explanation |
| :--- | :--- | :--- |
| Activation | ELU | Activation function. |
| Batch Size | 4 | Training data in one iteration. |
| Criterion | MSELoss | Loss function objective. |
| Early Stopping Patience | 300 | Epochs to wait for improvement. |
| Epochs | 50 | Total Number of Epochs. |
| Hidden Channels | 16 | Depth of the Network. |
| In Channels | 1 | Input feature map depth. |
| Kernel Size | 11 | The size of the kernel. |
| Learning Rate | 5e-05 | Optimizer step size. |
| Num Hidden Layers | 3 | Network depth. |
| Out Channels | 2 | Number of learned filters. |
| Output Activation | None | Final layer activation. |
| Padding Mode | zeros | Boundary condition strategy. |
| Random Seed | 1 | Torch Seed. |
| Random Split | False | Wheter to split data randomly. |
| Train Ratio | 0.6 | Training/Validation split proportion. |
| Use Bias | True | Learnable additive offset. |
| Use Early Stopping | True | Stop when model stops converging. |
| Use Lr Scheduler | True | Dynamic LR adjustment per epoch. |
| Weight Decay | 0.0001 | Regularization coefficient. |


In [ ]:
def format_dict_to_md(d, level=0):
    lines = []
    for k, v in d.items():
        indent = "  " * level
        if isinstance(v, dict) and v.get("min") or v.get("max"):
            lines.append(f"{indent}* **{k.capitalize()}**: From {v['min']:.4f} to {v['max']:.4f}")
        elif isinstance(v, dict):
            lines.append(f"{indent}* **{k.capitalize()}**:")
            lines.extend(format_dict_to_md(v, level + 1))
        else:
            lines.append(f"{indent}* **{k}**: {v}")
    return lines

md_text = "The input data was then normalized between 0 and 1 based on those ranges:\n\n"
md_text += "\n".join(format_dict_to_md(min_max_yaml))
display(Markdown(md_text))

In [ ]:
md_text = "# Evaluation\n"
md_text += "## Loss Graphs\n"
md_text += f"The following figure shows the training, validation and test loss over the {num_epochs} epochs the model was trained on.\n\n"
md_text += "![Train and test losses](losses.png)\n\n"
md_text += f"The best losses achieved during training (MSE on normalized data) were {best_train_loss:.4g} on the training data, "
md_text += f"{best_test_loss:.4g} on the test data and {best_val_loss:.4g} on the validation data.\n\n"
md_text += "## Performance Metrics\n"
md_text += "The table below summarizes the error metrics for the velocity components across all data splits on the denormalized fields (in original velocity units).\n\n"
md_text += "| Metric | Field | Training | Test | Validation |\n"
md_text += "| :--- | :--- | :--- | :--- | :--- |\n"
md_text += f"| **Mean absolute error** | u | {mae_u_train:.3g} | {mae_u_test:.3g} | {mae_u_val:.3g} |\n"
md_text += f"| | v | {mae_v_train:.3g} | {mae_v_test:.3g} | {mae_v_val:.3g} |\n"
md_text += f"| **Root mean square error** | u | {rmse_u_train:.3g} | {rmse_u_test:.3g} | {rmse_u_val:.3g} |\n"
md_text += f"| | v | {rmse_v_train:.3g} | {rmse_v_test:.3g} | {rmse_v_val:.3g} |\n"

display(Markdown(md_text))

In [ ]:
plot_dir = Path("plots") / "interpolation"

md_text = "# Predictions\n"
md_text += "We expect the model to perform better within the data range it was trained on (interpolation) "
md_text += "and perform worse on data it has not seen yet (extrapolation). So we differentiate between those.\n\n"
md_text += "## Interpolation\n\n"

error_plots = []
quiver_plots = []
prediction_plots = []

number = len(sorted(plot_dir.iterdir()))
md_text += f"Below are {number} predictions within the data range the model was trained on.\n\n"

for plot_subdir in sorted(plot_dir.iterdir()):
    for plot in reversed(sorted(plot_subdir.iterdir())):
        if plot.is_file():
            if "prediction" in plot.name and not "quiver" in plot.name:
                continue
            md_text += f"![plot]({plot})\n\n"     

display(Markdown(md_text))

In [ ]:
plot_dir = Path("plots") / "extrapolation"
error_plots = []
quiver_plots = []
prediction_plots = []

md_text = "# Extrapolation\n"
md_text += "The extrapolation section shows how good the model is at generalizing. We differentiate between three different extrapolations:\n\n"
md_text += "1. Other positive flow speeds\n"
md_text += "2. Other resolutions\n"
md_text += "3. Negative flow speeds/Flow at another boundary\n\n"
md_text += "We mostly care about the first and second case, because the third can be easily turned into the first by rotating/flipping the input.\n\n"
md_text += "### Other Flow Speeds\n\n"

for plot_subdir in sorted(plot_dir.iterdir()):
    if "flow_speed" in plot_subdir.name and not "negative" in plot_subdir.name:
        for plot in sorted(plot_subdir.iterdir()):
            md_text += f"![plot]({plot})\n\n"   

md_text += "### Other Resolutions\n\n"

for plot_subdir in sorted(plot_dir.iterdir()):
    if "resolution" in plot_subdir.name:
        for plot in sorted(plot_subdir.iterdir()):
            md_text += f"![plot]({plot})\n\n"   

md_text += "### Other Boundaries\n\n"

for plot_subdir in sorted(plot_dir.iterdir()):
    if "negative" in plot_subdir.name or "boundary" in plot_subdir.name:
        for plot in sorted(plot_subdir.iterdir()):
            md_text += f"![plot]({plot})\n\n"   

display(Markdown(md_text))